### Advent of Code 2021 | Day 4 | Part 1

In [1]:
from collections import Counter
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# Read in data
input = []
with open('input.txt') as f:
    for line in f:
        input.append(line.replace("\n", ""))

In [3]:
# Extract random number draw
rand_num_draw = input[0].split(',')
input.pop(0)

# Convert the string #'s to the int dtype
rand_num_draw = [int(item) for item in rand_num_draw]

In [4]:
# Remove leading empty line
input.pop(0)

# Extract bingo boards
boards = []
binary_boards = []
board = []
binary_board = []

for line in input:
    # If we encounter an empty line, then add the add the current board/binary_board to boards/binary_boards and reset board & binary_board to empty lists
    if line == '':
        boards.append(board)
        binary_boards.append(binary_board)
        board = []
        binary_board = []
        continue
        
    # Replace any double-spaces in the line with single-spaces and also strip leading & trailing spaces
    # Add the cleaned-up line to the current board
    line = line.replace('  ', ' ').strip().split(' ')
    
    # Convert the string #'s to the int dtype
    line = [int(item) for item in line]
    
    # Add the current line to the current board
    board.append(line)
    
    # Add a line of False values to binary_board
    binary_board.append([False, False, False, False, False])

In [5]:
# Create boards
board_list_of_dfs = []
for i, board in enumerate(boards):
    board_list_of_dfs.append([i, pd.DataFrame(board)])
    
# Create binary boards
binary_board_list_of_dfs = []
for i, binary_board in enumerate(binary_boards):
    binary_board_list_of_dfs.append([i, pd.DataFrame(binary_board)])

In [6]:
# Execute an iteration of the loop for every random number
progress_bar = tqdm(total = len(rand_num_draw))
try:
    for rand_i, rand in enumerate(rand_num_draw):
        
        # Search through every board for the random number
        for i, board_df in enumerate(board_list_of_dfs):
            # Extract the row index(s) & column index(s) when the random number is found
            ri, ci = np.where(board_df[1] == rand)
            
            # Actions to take if the random number is found
            if (len(ri) + len(ci)) > 0:
                # Convert the board index/row index(s)/column index(s) into coord style tuples
                bi_ri_ci = list(zip(itertools.repeat(i), ri, ci))
                
                # Plot the bingo hits
                for coord in bi_ri_ci:
                    # Set random number hit to True on the current binary board
                    binary_board_list_of_dfs[coord[0]][1].at[coord[1], coord[2]] = True
                    
                    # Check whether any of the current binary board's rows have bingo (returns True or False)
                    bingo_rows = any(binary_board_list_of_dfs[coord[0]][1].all(axis = 1).to_list())

                    # Check whether any of the current binary board's columns have bingo (returns True or False)
                    bingo_columns = any(binary_board_list_of_dfs[coord[0]][1].all(axis = 0).to_list())

                    # If the board is found to have bingo in the rows or columns, then Call "BINGO!"
                    if bingo_rows or bingo_columns:
                        # Use the current binary board as a mask to sum only the unmarked numbers on the current board
                        sum_unmarked = board_list_of_dfs[coord[0]][1].where(~binary_board_list_of_dfs[coord[0]][1]).apply(pd.to_numeric, errors='ignore').sum().sum()
                        
                        # Multiply the sum by the current randum number
                        final_score = int(sum_unmarked) * int(rand)
                        print(f'Bingo called on random #{rand} for board #{binary_board_list_of_dfs[coord[0]][0]} - final score: {final_score}!')
                        
                        # Break out of all nested loops
                        raise StopIteration
        
        progress_bar.update(1)

except StopIteration:
    progress_bar.update((len(rand_num_draw) - rand_i))
    progress_bar.close()

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 217.39it/s]

Bingo called on random #60 for board #41 - final score: 46920!
